<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך הסבת שכבת קדסטר מבוסס קואורדינטות לבנק"ל מודרני 🗺️ 

In [ ]:
print('7. Coordinates Based Cadastre Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [1]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  7.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
טבלת מספרי גושים של קמ"ק

In [2]:
CBC_list = pd.read_excel(CNFG.Local + 'CoordBasedCadstreBlocksList.xlsx', usecols = ['BlockNumber','SubBlockNumber'])

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר

In [4]:
Blocks_layer = daskgpd.read_file(CNFG.Results, layer = 'Blocks', columns = ['Name', 'geometry'], npartitions = 4).compute()

<h1 style="text-align:right;font-size:100%">
טבלת מזהי גושי קדסטר

In [ ]:
Blocks_IDs = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])
Blocks_IDs = Blocks_IDs[Blocks_IDs['IsTax'] == 0].drop(columns = 'IsTax')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  7.b Cleaning data')

In [ ]:
Blocks_IDs.rename(columns = {'GlobalID' : 'BlockUniqueID'}, inplace = True)
Blocks = Blocks_layer.merge(Blocks_IDs, how = 'left', on = 'Name')

del [Blocks_IDs, Blocks_layer]

In [ ]:
print('  7.c Data conversion')

In [ ]:
CBC_list['Name'] = CBC_list['BlockNumber'].astype(str) + '/' + CBC_list['SubBlockNumber'].astype(str)

<h1 style="text-align:right;font-size:100%">
מיזוג מידע

In [ ]:
merge = Blocks.merge(CBC_list, on = 'Name', how = 'right')
merge = merge.drop_duplicates(keep = 'first')
merge = merge[~merge['geometry'].isna()]
merge = merge.groupby(['Name'], as_index = False).first()

del[CBC_list, Blocks]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  7.d Defining new Layer')

In [ ]:
new_columns =np.dtype([
                        ("geometry", gpd.array.GeometryDtype),       
                       #("GlobalID", str),
                        ("ProcessName", str),
                        ("ProcessType", int),
                        ("Status", int),
                        ("BlockUniqueID", str),
                        ("GeodeticNetwork", int),
                       #("SurveyorLicenseID", int),
                        ("DataSource", int),
                       #("PlanName", str),  
                       #("ORACLE_ID", int),
                       #("Shape_Length", float),
                       #("Shape_Area", float),
                        ])

CBC_Blocks = gpd.GeoDataFrame(data     = np.empty(0, dtype = new_columns),
                              geometry = 'geometry',
                              crs      = 2039)

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
CBC_Blocks['BlockUniqueID'] = merge['BlockUniqueID']

CBC_Blocks['ProcessType'] = 5

CBC_Blocks['ProcessName'] = 'קמ"ק ' + merge['Name']

CBC_Blocks['GeodeticNetwork'] = 3

CBC_Blocks['Status'] = 13   #מאושר

#CBC_Blocks['PlanName'] = None

#CBC_Blocks['SurveyorLicenseID'] = None

CBC_Blocks['DataSource'] = 5 #CAD

CBC_Blocks['geometry'] = merge['geometry']

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('  7.e Exporting results')

In [ ]:
CBC_Blocks.to_file(CNFG.Results, layer = 'PT5', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')